In [90]:
import networkx as nx
import seaborn as sns
from pathlib import Path
import torch
from DQN import DQN_agent_modular
from envs.GraphEnv.impnode import ImpnodeEnv
from DQN.train_dqn import train_dqn, fill_memory
from DQN.finetune_dqn import finetune_dqn
from DQN.test_and_compare import test_loop, hda
import matplotlib.pyplot as plt
import time
import mlflow
import  numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
import random
seed = 412
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.use_deterministic_algorithms(True,warn_only=True)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [92]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


RESULTS_BASEPATH_TEST_ws = 'results/finetune_experiments/cuda_0.001_100000_100_0.99_64_50000_500_1.0_0.05_5000_0.0001_4_2_32_64_1_3_(30, 50)_dw_nd_small-world_20240424134422/11000_model.pt'
RESULTS_BASEPATH_TEST_ba = 'results/finetune_experiments/cuda_0.001_100000_100_0.99_64_50000_500_1.0_0.05_5000_0.0001_4_2_32_64_1_3_(30, 50)_dw_nd_barabasi-albert_20240424124037/9000_model.pt'
RESULTS_BASEPATH_TEST_er = 'results/finetune_experiments/cuda_0.001_100000_100_0.99_64_50000_500_1.0_0.05_5000_0.0001_4_2_32_64_1_3_(30, 50)_dw_nd_erdos-renyi_20240424134434/20000_model.pt'

In [93]:
dqn_agent_test = DQN_agent_modular.DQNAgent(device=device,
                                    alpha=0.001,
                                    gnn_depth=4,
                                    state_size=2,
                                    hidden_size1=32,
                                    hidden_size2=64,
                                    action_size=1,
                                    discount=0.0,
                                    eps_max=0.0,
                                    eps_min=0.0,
                                    eps_step=0.0,
                                    memory_capacity=0,
                                    lr=0,
                                    mode='test')




In [99]:
dqn_agent_test.load_model('{}'.format(RESULTS_BASEPATH_TEST_ba))
subdir = 'data/synthetic_small_dataset/BA_val_30_50'
data_path = Path.cwd()/subdir

NUM_TEST_EPS = 1 # number of test episodes to run
 
env_test_val = ImpnodeEnv(anc='dw_nd', g_type='barabasi-albert', num_nodes=(30, 50), data_path=data_path, mode='test', max_removed_nodes=None)

# test the agent
actions2, reward_history2, ep_score_history2 = test_loop(env=env_test_val,
                                                         agent=dqn_agent_test,
                                                         NUM_TEST_EPS = NUM_TEST_EPS)

np.mean(ep_score_history2)

0.35268702651515166

In [97]:
dqn_agent_test.load_model('{}'.format(RESULTS_BASEPATH_TEST_ws))
subdir = 'data/synthetic_small_dataset/WS_val_30_50'
data_path = Path.cwd()/subdir

NUM_TEST_EPS = 100 # number of test episodes to run

env_test_val = ImpnodeEnv(anc='dw_nd', g_type='small-world', num_nodes=(30, 50), data_path=data_path, mode='test', max_removed_nodes=None)

# test the agent
actions2, reward_history2, ep_score_history2 = test_loop(env=env_test_val,
                                                         agent=dqn_agent_test,
                                                         NUM_TEST_EPS = NUM_TEST_EPS)

np.mean(ep_score_history2)

AssertionError: Env is terminated. Use reset()

In [89]:
dqn_agent_test.load_model('{}'.format(RESULTS_BASEPATH_TEST_er))
subdir = 'data/synthetic_small_dataset/ER_val_30_50'
data_path = Path.cwd()/subdir

NUM_TEST_EPS = 100 # number of test episodes to run

env_test_val = ImpnodeEnv(anc='dw_nd', g_type='erdos-renyi', num_nodes=(30, 50), data_path=data_path, mode='test', max_removed_nodes=None)

# test the agent
actions2, reward_history2, ep_score_history2 = test_loop(env=env_test_val,
                                                         agent=dqn_agent_test,
                                                         NUM_TEST_EPS = NUM_TEST_EPS)

np.mean(ep_score_history2)

0.19526946140795748